In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import LinearRegression, GBTRegressor, RandomForestRegressor, DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import Vectors
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from google.colab import files
import shutil

In [ ]:
# Iniciar una sesión de PySpark
spark = SparkSession.builder.appName("MortgagePrediction").getOrCreate()

In [ ]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")


In [ ]:
# Cargar los datos
file_path = "/content/drive/My Drive/clean_mortgage_data2.csv"
data = spark.read.csv(file_path, header=True, inferSchema=True)

data=data.orderBy(["YEAR", "MONTH"], ascending=[True, True])
# Mostrar las primeras filas
data.show(5)

+------+---------+------------+---------------+------+--------------------+--------------------+------+----+-------+-----+----------+-----------+-----------+----------+
|SOURCE|FREQUENCY|    SERIESID|       GEOLEVEL| GEOID|             GEONAME|              MARKET|PERIOD|YEAR|QUARTER|MONTH|SUPPRESSED|MORT_NUMBER|DOLLAR_MEAN|VOL_DOLLAR|
+------+---------+------------+---------------+------+--------------------+--------------------+------+----+-------+-----+----------+-----------+-----------+----------+
|  NMDB|  Monthly| PCT_CASHOUT|Census Division|   DMA|     Middle Atlantic|   Conforming Market|199801|1998|      1|    1|         0|       22.1|       20.2|    8928.4|
|  NMDB|  Monthly|PCT_AGE_LT25|Rural/Non-Rural| USARA| United States Rural| Conventional Market|199801|1998|      1|    1|         0|        3.7|        2.4|     177.6|
|  NMDB|  Monthly|  PCT_TWOBOR|Census Division|   DMA|     Middle Atlantic|       All Mortgages|199801|1998|      1|    1|         0|       62.3|       68.

In [ ]:
data = data.filter(F.col("MARKET").contains("Other Conventional Market"))


In [ ]:
# Realizar el agrupamiento y la agregación
data = data.groupBy("MONTH", "YEAR", "QUARTER").sum()



In [ ]:
# Para renombrar las columnas de los resultados de la agregación
data = data.withColumnRenamed("sum(MORT_NUMBER)", "MORT_NUMBER")
data = data.withColumnRenamed("sum(DOLLAR_MEAN)", "DOLLAR_MEAN")
data = data.withColumnRenamed("sum(VOL_DOLLAR)", "VOL_DOLLAR")

In [ ]:
# Supongamos que df es tu DataFrame en PySpark
data = data.withColumn(
    "TIMESTAMP",
    F.to_date(
        F.concat_ws("-", F.col("YEAR"), F.col("MONTH"), F.lit(1)),
        "yyyy-MM-dd"
    )
)


In [ ]:
data

DataFrame[MONTH: int, YEAR: int, QUARTER: int, sum(PERIOD): bigint, sum(YEAR): bigint, sum(QUARTER): bigint, sum(MONTH): bigint, sum(SUPPRESSED): bigint, MORT_NUMBER: double, DOLLAR_MEAN: double, VOL_DOLLAR: double, TIMESTAMP: date]

In [ ]:
data.show(5)

+-----+----+-------+-----------+---------+------------+----------+---------------+------------------+------------------+--------------------+----------+
|MONTH|YEAR|QUARTER|sum(PERIOD)|sum(YEAR)|sum(QUARTER)|sum(MONTH)|sum(SUPPRESSED)|       MORT_NUMBER|       DOLLAR_MEAN|          VOL_DOLLAR| TIMESTAMP|
+-----+----+-------+-----------+---------+------------+----------+---------------+------------------+------------------+--------------------+----------+
|    1|1998|      1|  556245984|  5562432|        2784|      2784|              0|103121.40000000007| 317456.8999999995| 9.992645416000001E8|1998-01-01|
|    2|1998|      1|  556248768|  5562432|        2784|      5568|              0|104323.30000000008|370641.30000000005|1.2476062585999997E9|1998-02-01|
|    3|1998|      1|  556251552|  5562432|        2784|      8352|              0|105092.79999999986| 452450.6000000013|1.7538765800000055E9|1998-03-01|
|    4|1998|      2|  556254336|  5562432|        5568|     11136|              0|

In [ ]:
# Selección de columnas relevantes
columns = [ 'MORT_NUMBER', 'DOLLAR_MEAN','VOL_DOLLAR','TIMESTAMP']
data = data.select(*columns)


In [ ]:
data = data.orderBy("TIMESTAMP")

In [ ]:
data.show(5)

+------------------+------------------+--------------------+----------+
|       MORT_NUMBER|       DOLLAR_MEAN|          VOL_DOLLAR| TIMESTAMP|
+------------------+------------------+--------------------+----------+
|103121.40000000007| 317456.8999999995| 9.992645416000001E8|1998-01-01|
|104323.30000000008|370641.30000000005|1.2476062585999997E9|1998-02-01|
|105092.79999999986| 452450.6000000013|1.7538765800000055E9|1998-03-01|
|104642.40000000005| 439381.0000000008|1.6369086298000038E9|1998-04-01|
|104719.50000000016| 435163.9000000002|1.5966773100000017E9|1998-05-01|
+------------------+------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
data.describe().show()

+-------+------------------+------------------+--------------------+
|summary|       MORT_NUMBER|       DOLLAR_MEAN|          VOL_DOLLAR|
+-------+------------------+------------------+--------------------+
|  count|               312|               312|                 312|
|   mean|132191.72083333333| 546117.8714743594|1.8466055602839735E9|
| stddev|15084.050106990711| 294699.3085584442|1.7664352344736922E9|
|    min|103013.50000000013|202347.80000000028| 6.318693759999977E8|
|    max|162684.30000000005| 1556480.800000001| 9.683853394399994E9|
+-------+------------------+------------------+--------------------+



In [ ]:
# Convertir la columna TIMESTAMP en un formato numérico
data = data.withColumn("TIMESTAMP_numeric", F.datediff(data["TIMESTAMP"], F.lit("1970-01-01")))

In [ ]:
# Crear las características usando la columna TIMESTAMP_numeric
assembler = VectorAssembler(inputCols=["TIMESTAMP_numeric"], outputCol="features")
data = assembler.transform(data)

In [ ]:
total_records = data.count()

In [ ]:
total_records

312

In [ ]:
train_data = data.limit(int(total_records * 0.8))  # 80% de los registros
test_data = data.subtract(train_data)

In [ ]:
train_data.show()

+------------------+------------------+--------------------+----------+-----------------+---------+
|       MORT_NUMBER|       DOLLAR_MEAN|          VOL_DOLLAR| TIMESTAMP|TIMESTAMP_numeric| features|
+------------------+------------------+--------------------+----------+-----------------+---------+
|103121.40000000007| 317456.8999999995| 9.992645416000001E8|1998-01-01|            10227|[10227.0]|
|104323.30000000008|370641.30000000005|1.2476062585999997E9|1998-02-01|            10258|[10258.0]|
|105092.79999999986| 452450.6000000013|1.7538765800000055E9|1998-03-01|            10286|[10286.0]|
|104642.40000000005| 439381.0000000008|1.6369086298000038E9|1998-04-01|            10317|[10317.0]|
|104719.50000000016| 435163.9000000002|1.5966773100000017E9|1998-05-01|            10347|[10347.0]|
|105209.20000000011| 478007.8999999996| 1.823710642999997E9|1998-06-01|            10378|[10378.0]|
|104833.50000000025|456500.20000000065| 1.689913690799999E9|1998-07-01|            10408|[10408.0]|


In [ ]:
test_data.show()

+------------------+------------------+--------------------+----------+-----------------+---------+
|       MORT_NUMBER|       DOLLAR_MEAN|          VOL_DOLLAR| TIMESTAMP|TIMESTAMP_numeric| features|
+------------------+------------------+--------------------+----------+-----------------+---------+
| 142926.6000000004| 420309.0999999987| 8.983399066000016E8|2018-10-01|            17805|[17805.0]|
|141155.59999999957|381723.70000000106| 8.426970349999983E8|2018-11-01|            17836|[17836.0]|
| 140236.5999999996|357773.89999999857| 8.028151040000001E8|2018-12-01|            17866|[17866.0]|
|141017.69999999984|328128.09999999905| 7.452390238000002E8|2019-01-01|            17897|[17897.0]|
|142895.00000000026| 334656.0999999996| 7.569825156000009E8|2019-02-01|            17928|[17928.0]|
| 143311.5999999998| 388671.5999999988| 8.367293964000016E8|2019-03-01|            17956|[17956.0]|
|144650.99999999994|460558.39999999816| 9.620435490000011E8|2019-04-01|            17987|[17987.0]|


In [ ]:
# Crear un modelo de regresión lineal para predecir DOLLAR_MEAN
lr = LinearRegression(featuresCol="features", labelCol="DOLLAR_MEAN")

In [ ]:
# Entrenar el modelo
lr_model = lr.fit(train_data)

# Realizar predicciones sobre el conjunto de prueba
predictions = lr_model.transform(test_data)

In [ ]:
# Evaluar el modelo usando RMSE (Root Mean Squared Error)
evaluator = RegressionEvaluator(labelCol="DOLLAR_MEAN", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"RMSE: {rmse}")

RMSE: 286828.4259983004


In [ ]:
# Obtener las predicciones y los valores reales para compararlos
predicted_values = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()
real_values = predictions.select("DOLLAR_MEAN").rdd.flatMap(lambda x: x).collect()


In [ ]:
real_values

[420309.0999999987,
 381723.70000000106,
 357773.89999999857,
 328128.09999999905,
 334656.0999999996,
 388671.5999999988,
 460558.39999999816,
 545081.9999999992,
 522096.19999999925,
 554839.0000000001,
 559816.7999999969,
 564910.6999999981,
 626109.2999999984,
 577255.3999999997,
 597364.3000000007,
 504798.7999999989,
 470553.49999999884,
 531555.2999999976,
 561237.8999999992,
 570990.499999998,
 524054.8000000007,
 549032.7999999981,
 559356.2999999984,
 499093.99999999854,
 600850.2999999986,
 558537.199999998,
 582712.8999999969,
 532498.1000000011,
 610106.8999999987,
 794155.5999999964,
 798929.299999997,
 778150.4999999986,
 816199.2999999961,
 766999.7999999982,
 785053.6999999973,
 754727.399999997,
 803383.0999999975,
 782216.7999999999,
 802331.7999999977,
 629100.8999999978,
 630702.9999999994,
 769459.2000000001,
 741451.2999999976,
 697970.100000001,
 695395.3999999986,
 570837.7999999989,
 548099.8999999989,
 476796.8999999999,
 441426.2000000002,
 384058.0999999982

In [ ]:
predicted_values

[346619.64205796435,
 344977.7220471398,
 343388.7671979547,
 341746.84718713013,
 340104.92717630544,
 338621.90265039937,
 336979.9826395748,
 335391.0277903897,
 333749.10777956515,
 332160.15293037996,
 330518.2329195554,
 328876.3129087308,
 327287.35805954575,
 325645.4380487212,
 324056.483199536,
 322414.5631887114,
 320772.64317788684,
 319236.65349034127,
 317594.7334795167,
 316005.7786303315,
 314363.85861950694,
 312774.90377032186,
 311132.9837594973,
 309491.0637486726,
 307902.10889948753,
 306260.18888866296,
 304671.2340394779,
 303029.3140286533,
 301387.3940178286,
 299904.36949192255,
 298262.449481098,
 296673.4946319129,
 295031.57462108834,
 293442.61977190315,
 291800.6997610786,
 290158.779750254,
 288569.82490106893,
 286927.90489024424,
 285338.9500410592,
 283697.0300302346,
 282055.11001941,
 280572.08549350395,
 278930.16548267927,
 277341.2106334942,
 275699.2906226696,
 274110.33577348455,
 272468.41576266,
 270826.4957518353,
 269237.5409026502,
 26759

In [ ]:
lr_model.save("/content/lr_model5")

In [ ]:
files.download("/content/lr_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>